### Silver Layer: Process results data

Read from bronze/raw layer (parquet) and transform for silver/processed layer.

In [ ]:
dbutils.widgets.text("p_data_source", "")
v_data_source = dbutils.widgets.get("p_data_source")

In [ ]:
from formula1.formula1_constants import raw_folder_path, processed_folder_path

In [ ]:
from formula1.formula1_utils import add_ingestion_date

##### Step 1 - Read parquet from bronze/raw layer

In [ ]:
results_df = spark.read.parquet(f"{raw_folder_path}/results")

##### Step 2 - Rename columns and add new columns

In [ ]:
results_with_columns_df = results_df \
    .withColumnRenamed("resultId", "result_id") \
    .withColumnRenamed("raceId", "race_id") \
    .withColumnRenamed("driverId", "driver_id") \
    .withColumnRenamed("constructorId", "constructor_id") \
    .withColumnRenamed("positionText", "position_text") \
    .withColumnRenamed("positionOrder", "position_order") \
    .withColumnRenamed("fastestLap", "fastest_lap") \
    .withColumnRenamed("fastestLapTime", "fastest_lap_time") \
    .withColumnRenamed("fastestLapSpeed", "fastest_lap_speed")

In [ ]:
results_with_ingestion_date_df = add_ingestion_date(results_with_columns_df)

##### Step 3 - Drop the unwanted column

In [ ]:
from pyspark.sql.functions import col

In [ ]:
results_final_df = results_with_ingestion_date_df.drop(col("statusId"))

##### Step 4 - Write to output to silver/processed layer in parquet format

In [ ]:
results_final_df.write.mode("overwrite").partitionBy('race_id').parquet(f"{processed_folder_path}/results")